In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import json
import pandas as pd

from bellek.jerx.reward.llm import make_question_answer_func
from bellek.utils import set_seed, jprint
from bellek.musique.baseline import benchmark

set_seed(89)

In [3]:
def silence(exc_cls):
    def decorator(func):
        def wrapper(*args, **kwargs):
            try:
                return func(*args, **kwargs)
            except exc_cls as e:
                return None
        return wrapper
    return decorator

In [4]:
df = pd.read_json('../../data/generated/musique-evaluation/dataset.jsonl', orient='records', lines=True)
print(len(df))
df.head()

100


,id,paragraphs,question,question_decomposition,answer,answer_aliases,answerable,answers
0,2hop__131818_161450,"[{'idx': 0, 'title': 'Maria Carrillo High Scho...",Where is the Voshmgir District located?,"[{'id': 131818, 'question': 'Which state is Vo...",in the north-east of the country south of the ...,"[Caspian Sea, in the north-east of the country...",True,"[Caspian Sea, in the north-east of the country..."
1,2hop__444265_82341,"[{'idx': 0, 'title': 'Ocala, Florida', 'paragr...",In what part of Florida is Tom Denney's birthp...,"[{'id': 444265, 'question': 'Tom Denney >> pla...",in Northern Florida,"[in Northern Florida, Northern Florida]",True,"[in Northern Florida, Northern Florida]"
2,2hop__711946_269414,"[{'idx': 0, 'title': 'Wild Thing (Tone Lōc son...",What record label is the performer who release...,"[{'id': 711946, 'question': 'All Your Faded Th...",Kill Rock Stars,[Kill Rock Stars],True,[Kill Rock Stars]
3,2hop__311931_417706,"[{'idx': 0, 'title': 'The Main Attraction (alb...",What record label does the performer of Emotio...,"[{'id': 311931, 'question': 'Emotional Rain >>...",Attic Records,"[Attic, Attic Records]",True,"[Attic, Attic Records]"
4,2hop__809785_606637,"[{'idx': 0, 'title': 'The Main Attraction (alb...",What record label does the performer of Advent...,"[{'id': 809785, 'question': 'Adventures in You...",Secret City Records,[Secret City Records],True,[Secret City Records]


In [5]:
COMPLETION_KWARGS={"temperature": 0.0, "max_tokens": 1024}

## gpt-3.5-turbo

In [6]:
# qa_func = make_question_answer_func("gpt-3.5-turbo", completion_kwargs=COMPLETION_KWARGS)

In [7]:
# df, scores = benchmark(df, qa_func, only_supporting=False)
# jprint(scores)

In [8]:
# df, scores = benchmark(df, qa_func, only_supporting=True)
# jprint(scores)

## gpt-4-turbo

In [9]:
# qa_func = make_question_answer_func("gpt-4o", completion_kwargs=COMPLETION_KWARGS)

In [10]:
# df, scores = benchmark(df, qa_func, only_supporting=False)
# jprint(scores)

In [11]:
# df, scores = benchmark(df, qa_func, only_supporting=True)
# jprint(scores)

## llama-3-70b

In [12]:
qa_func = make_question_answer_func("llama-3-70b-tgi", completion_kwargs=COMPLETION_KWARGS)

In [13]:
df, scores = benchmark(df, qa_func, only_supporting=False, ignore_errors=True)
jprint(scores)

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.49,
  "f1": 0.5807157287157286,
  "fuzzy_match": 0.58
}


In [14]:
df, scores = benchmark(df, qa_func, only_supporting=True, ignore_errors=True)
jprint(scores)

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.59,
  "f1": 0.6924935064935065,
  "fuzzy_match": 0.69
}


## Inspect

In [15]:
def present_example(example, predicted_answer):
    text = "\n\n".join([p["paragraph_text"] for p in example['paragraphs']])
    print("="*80)
    print("Question:", example["question"])
    print("Reference Answer:", example['answer'])
    print("Predicted Answer:", predicted_answer)
    print("-"*80)
    print("Paragraphs")
    print(text)